# Vérification du codage de la distance au bord le plus proche

In [ ]:
from trustutils import run
import os
from string import Template

run.introduction("C. Reiss","29/04/2022")
run.description(r'On teste les fonctions qui permet de remplir le tableau de distance au bord le plus proche et de le post-traiter. Pour determiner cette distance, l\'algorithme charge l\'ensemble des coordonnees des centres des faces de bord, ainsi que les sommets et les aretes. Il echange ces informations entre processeurs. Il utilise ensuite la fonction findClosestTupleId de MEDcoupling pour faire une recherche efficace. Cette methode n\'est pas exacte pour les maillages deformes ou de tetraedres car la projection du centre d\'un element de volume sur une face peut ne pas etre pile sur un centre de gravite de face ou sur un sommet. Pour que les resultats soient exacts dans les elements de bord, on fait une boucle sur ceux-ci en fin d\'iteration.')

run.TRUST_parameters()

In [ ]:
schema = { "PolyMAC" : " PolyMAC  dis ",
            "PolyMAC_P0": " PolyMAC_P0  dis option_PolyMAC_P0 {  } ",
            "PolyMAC_P0P1NC": " PolyMAC_P0P1NC  dis ",
            "VDF" : " vdf  dis ", 
            "VEF" : " VEFPreP1B dis " }

maillage = [ 0, 0, 
             {  "PolyMAC" : " ",
                "PolyMAC_P0": " ",
                "PolyMAC_P0P1NC": " ",
                "VDF" : " ", 
                "VEF" : " trianguler_h dom " },
             { "PolyMAC" : " polyedriser dom ",
                "PolyMAC_P0": " polyedriser dom ",
                "PolyMAC_P0P1NC": " polyedriser dom ",
                "VDF" : "  ", 
                "VEF" : " tetraedriser_homogene dom " }
            ] 


In [ ]:
force_recalculation = True

number_of_partitions = 8

def GenerateInputFile(name,substitutions_dict, dim):
    with open(f"build/jdd_{dim}D.data", "r") as file: 
        filedata = Template(file.read())
    result = filedata.substitute(substitutions_dict)
    with open(f"build/{name}.data", "w") as file:
        file.write(result)

if (force_recalculation) :
    run.reset()
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)

    for s in schema.keys():
        for d in [2,3] :
            for p in ["seq", "par"]:
                name = f"{s}_{d}D_{p}"
                os.system(f'mkdir build/{name}') 
                substitutions_dict_2D = {  "schema" : schema[s], "maillage" : maillage[d][s] }
                GenerateInputFile(name,substitutions_dict_2D, d)
                if p == "seq" :
                    os.system(f'mv build/{name}.data build/{name}')
                    run.addCase(f"{name}", f"{name}.data")
                elif p == "par" :
                    os.system(f'mv build/{name}.data build/{name} ; cd build/{name};  make_PAR.data {name} {number_of_partitions}; cd ../..')
                    run.addCase(f"./{name}", f"PAR_{name}.data",nbProcs=number_of_partitions)

    run.addCase(".","jdd_triangle.data")

    run.printCases()
    run.runCases()

## Affichage de du lata de la distance a la paroi en 2D dans une geometrie compliquee sur des rectangles ou triangles suivant le schema

In [ ]:
from trustutils import visit
from trustutils.jupyter import plot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

for s in schema.keys():
    for d in [2] :
        for p in ["seq", "par"]:
            name = f"{s}_{d}D_{p}"
            par = ""
            if (p == "par") : par = "PAR_"
            fig=visit.showField(f"{name}/{par}{name}.lata","Pseudocolor","DISTANCE_PAROI_GLOBALE_ELEM_dom",iteration=0)



## Affichage de comparaison de sondes en 2D et en 3D sur la geometrie precedente contre la solution analytique sur des rectangles, triangles ou tetrahedres suivant le schema

In [ ]:
""" On affiche les distances le long de sondes pour tous les schemas en 2D, 3D, parallele et sequentiel. Le nom des positions depend de l'endroit du maillage dans lequelle elles ont ete tirees. Il y a moins de points dans le volume en 3D pour garder un temps de generation de la fiche correct. Les ecarts en VEF sont probablement dus au decalage des centres de gravite des mailles induits par les caractere tetrahedrique/triangulaire (voir ci-dessous)."""


fig = plt.figure(figsize=(12, 8))

plt.subplots_adjust(hspace= 0.3)

dic_subplots = { "NORD" : fig.add_subplot(2, 6, (1,2)) ,
                 "OUEST": fig.add_subplot(2, 6, (3, 4)),
                 "SUD1" : fig.add_subplot(2, 6, (5, 6)),
                 "SUD2" : fig.add_subplot(2, 6, (7,9)) ,
                 "EST"  : fig.add_subplot(2, 6, (10,12)) 
                }

absc = { "NORD" : np.linspace(-.975, .975, 40) ,
         "OUEST": np.linspace(-1.9, -1.1, 5),
         "SUD1" : np.linspace(-1.9, -1.1, 5),
         "SUD2" : np.linspace(.025, 4.975, 100),
         "EST"  : np.linspace(2**0.5*2.95, 0, 60)
        }

absc = { "NORD" : np.linspace(-.975, .975, 40) ,
         "OUEST": np.linspace(-1.9, -1.1, 5),
         "SUD1" : np.linspace(-1.9, -1.1, 5),
         "SUD2" : np.linspace(.025, 4.975, 100),
         "EST"  : np.linspace(2**0.5*2.95, 0, 60)
        }

dict_marker = { "PolyMAC" : ":", "PolyMAC_P0": ":", "PolyMAC_P0P1NC": ":", "VDF" : "--", "VEF" : "-" }
dict_linesize = { "PolyMAC" : 4, "PolyMAC_P0": 4, "PolyMAC_P0P1NC": 4, "VDF" : 3, "VEF" : 2 }

tab_color = [ 0, 0, {"seq" : "blue", "par" : "green" }, {"seq" : "red", "par" : "orange" }]

xe_loc = 4.975-absc["EST"]/2**.5
ye_loc = -.975+absc["EST"]/2**.5

ana = { "NORD" : 0.5 - absc["NORD"] + absc["NORD"] ,
         "OUEST": .5-np.abs(absc["OUEST"]+1.5),
         "SUD1" : np.abs(absc["SUD1"]+1),
         "SUD2" : np.minimum( 5-absc["SUD2"] , ( np.maximum(absc["SUD2"]-2, 0)**2 + 0.9**2)**.5 ),
         "EST"  : np.minimum( ((xe_loc-5)**2+(ye_loc+1)**2)**.5 , xe_loc-2)
        }

dic_subplots["NORD"].set_ylim(.4, .8)

for loc in dic_subplots.keys():
    for s in schema.keys():
        for d in [2,3] :
            for p in ["seq", "par"]:
                name = f"{s}_{d}D_{p}"
                par = ""
                if (p == "par") : par = "PAR_"
                dist = np.array(plot.loadText(f"{name}/{par}{name}_{loc}.son")[1::])
                dic_subplots[loc].plot(absc[loc], dist, dict_marker[s], linewidth = dict_linesize[s], color = tab_color[d][p] , label = name)
    dic_subplots[loc].plot(absc[loc], ana[loc], color = "k", label = "Analytique")
    dic_subplots[loc].legend(fontsize = 6, ncol = 2)
    dic_subplots[loc].set_xlabel("x-position")
    dic_subplots[loc].set_ylabel("distance from wall")
    dic_subplots[loc].set_title(loc)

plt.tight_layout()
plt.savefig("Distance_paroi_tous_schemas.pdf")



## Affichage de la distance sur des tetras un peu tordus en proche paroi

In [ ]:
"""On regarde les resultats dans un maillage triangulaire 2D et on les compare avec la theorie. On tire une sonde la les mailles proches de la paroi (Y=0.5). On voit que la solution exacte est trouvee dans l'element de bord grace a la boucle supplementaire sur ceux-ci, mais pas dans l'element suivant pour lequel la distance calculee est un peu elevee."""

fig=visit.showField("jdd_triangle.lata","Pseudocolor","DISTANCE_PAROI_GLOBALE_ELEM_dom")

In [ ]:
d = np.array(plot.loadText("jdd_triangle_DIST.son")[1::])
x = np.linspace(0.5, 9.5, 20)

d_min = np.linspace(0.333333, 0.333333, 20)
d_max = np.linspace(0.666666, 0.666666, 20)

plt.plot(x, d_min, 'b', linewidth = 2, label = 'Possible low value')
plt.plot(x, d_max, 'g', linewidth = 2, label = 'Possible high value')
plt.plot(x,     d, 'or', label = 'trust output')
plt.xlabel("position on the probe")
plt.ylabel("distance to the edge")
plt.legend()

plt.ylim(0, 1)

plt.show()